Import the libraries

In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### EXTRACT THE SUBCATEGORIES LINKS IN FAQ WEB PAGE

In [92]:
# define the url 
url = pd.read_csv('C:\\Masaüstü\\DATA INFORMATICS courses\\DI502 - DI Project\\kod\\courses_link.txt', header = None)

In [93]:
url.head(1)

,0
0,https://catalog.metu.edu.tr/prog_courses.php?p...


In [94]:
url_list=url[0].tolist()

In [97]:
# define the request 
courses = []
for i in url_list:
    session_req = requests.session()
    response  = session_req.get(i)

    # parse the url context
    soup = BeautifulSoup(response.text, "html.parser")
    # and find the all extensions of the FAQ's links 
    network_section = soup.find("table").find_all("a")

    # find the all courses links for each undergraduate department
    subcategories_links = []
    for k in network_section:
        _temp_link = "https://catalog.metu.edu.tr/" + str(k.get("href"))
        _title = k.text
        subcategories_links.append((_temp_link,_title))

    # NOW GET THE COURSE INFORMATION FROM THE SUBCATEGORIES
    for x in subcategories_links:
        # define the request 
        session_req = requests.session()
        response = session_req.get(x[0])

        _temp_link = x[0]
        _temp_title = x[1]
        _temp_soup = BeautifulSoup(response.text,"html.parser")

        #find course objectives link, because there is a render 
        _course_objectives_link =[]
        for j in _temp_soup.find_all("iframe"):
            _course_objectives_link.append(j.get("src"))

        _temp_response = session_req.get(_course_objectives_link[0])
        _temp_soup2 = BeautifulSoup(_temp_response.text, "html.parser")
        _temp_course_objective = _temp_soup2.find("p")
        if _temp_course_objective:
            _temp_courseobjective = _temp_course_objective.text
        else: 
            _temp_courseobjective = "No objective"
        courses.append({"link": _temp_link,"course": _temp_title, "course_objective": _temp_courseobjective})

In [101]:
df = pd.DataFrame(courses)

In [103]:
df.to_excel(r'C:\Users\merve\Downloads\courses_eng.xlsx', index=False)